In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import numba

In [18]:
def getPositionsCharges(boxSize: int, charge: float = 1.0):
    """
    Generate positions and charges for atoms in a cubic lattice.

    Parameters:
    boxSize (int): The size of the box (number of atoms along one axis).
    charge (float): The magnitude of the charge for each atom (default is 1.0).

    Returns:
    positions (np.ndarray): Array of atom positions.
    charges (np.ndarray): Array of charges corresponding to each position.
    """
    mgrid = np.mgrid[0:boxSize, 0:boxSize, 0:boxSize].reshape(3, -1).T
    positions = mgrid + 0.5
    charges = charge * (-1) ** (mgrid.sum(axis=1))
    return positions.astype(np.float64), charges.astype(np.float64)


PA = 3.97886080735226
erfcCoeffs = np.array(
    [
        2.75374741597376782e-1,
        4.90165080585318424e-1,
        7.74368199119538609e-1,
        1.07925515155856677,
        1.31314653831023098,
        1.37040217682338167,
        1.18902982909273333,
        0.805276408752910567,
        0.357524274449531043,
        0.0166207924969367356,
        -0.119463959964325415,
        -0.0838864557023001992,
        0.00249367200053503304,
        0.0390976845588484035,
        0.0161315329733252248,
        -0.0133823644533460069,
        -0.0127223813782122755,
        0.00383335126264887303,
        0.00773672528313526668,
        -0.000870779635317295828,
        -0.00396385097360513500,
        0.000119314022838340944,
        0.00127109764952614092,
    ]
)


@numba.vectorize([numba.float64(numba.float64)])
def erfc(x):
    """
    Approximate the complementary error function using a rational approximation.

    Parameters:
    x (float): The input value.

    Returns:
    y (float): The approximate value of erfc(x).
    """
    t = PA / (PA + np.abs(x))
    u = t - 0.5
    uPowers = u ** np.arange(23)
    y = np.sum(uPowers * erfcCoeffs)
    y *= t * np.exp(-(x**2))
    return y


@numba.vectorize([numba.float64(numba.float64, numba.float64)])
def wrap(x, boxSize):
    """
    Apply periodic boundary conditions to coordinate x.

    Parameters:
    x (float): The coordinate to be wrapped.
    boxSize (float): The size of the box.

    Returns:
    float: The wrapped coordinate.
    """
    return x - boxSize * np.round(x / boxSize)


@numba.njit
def realSpace(positions: np.ndarray, charges: np.ndarray, boxSize: float, alpha: float) -> np.float64:
    """
    Compute the real-space contribution to the Ewald summation.

    Parameters:
    positions (np.ndarray): Array of atom positions.
    charges (np.ndarray): Array of charges.
    boxSize (float): The size of the simulation box.
    alpha (float): The Ewald damping parameter.

    Returns:
    float: The real-space energy per atom.
    """
    energy = 0.0
    n = positions.shape[0]
    halfBoxSq = 0.25 * boxSize**2
    for i in range(n - 1):
        for j in range(i + 1, n):
            dr = positions[i] - positions[j]

            # Apply periodic boundary conditions
            dr = wrap(dr, boxSize)
            r2 = np.dot(dr, dr)
            if r2 < halfBoxSq:
                r = np.sqrt(r2)

                # Accumulate pairwise energy using the error function
                energy += charges[i] * charges[j] * erfc(alpha * r) / r
    return energy / n


def fourierSpace(
    positions: np.ndarray, charges: np.ndarray, boxSize: float, alpha: float, numberOfWaveVectors: int
) -> np.float64:
    """
    Compute the Fourier-space contribution to the Ewald summation.

    Parameters:
    positions (np.ndarray): Array of atom positions.
    charges (np.ndarray): Array of charges.
    boxSize (float): The size of the simulation box.
    alpha (float): The Ewald damping parameter.
    numberOfWaveVectors (int): The maximum number of wave vectors to consider in each direction.

    Returns:
    float: The Fourier-space energy per atom.
    """
    energy = 0.0
    numberOfAtoms = positions.shape[0]

    # Initialize arrays for exponentials e^{ikx}
    eik = np.ones((numberOfWaveVectors + 1, numberOfAtoms, 3), dtype=np.complex128)
    eik_xy = np.ones(numberOfAtoms, dtype=np.complex128)

    boxUnitCircle = 2.0 * np.pi / boxSize
    volume = boxSize**3

    # Compute scaled positions
    s = boxUnitCircle * positions

    eik[1] = np.cos(s) + np.sin(s) * 1j

    # Build higher order exponentials using recursion
    for k in range(2, numberOfWaveVectors + 1):
        eik[k] = eik[k - 1] * eik[1]

    # Triple loop over wave vectors kx, ky and kz
    for kx in range(numberOfWaveVectors + 1):
        # Energy for kx equals -kx, therefore we can compute once, but multiply by 2
        doubleForKX = 1.0 if kx == 0 else 2.0

        for ky in range(-numberOfWaveVectors, numberOfWaveVectors + 1):
            # Adjust the sign for negative ky
            eiky_temp = np.copy(eik[abs(ky), :, 1])
            eiky_temp.imag *= np.sign(ky)

            # Compute e^{i(kx x + ky y)}
            eik_xy = eik[kx, :, 0] * eiky_temp

            for kz in range(-numberOfWaveVectors, numberOfWaveVectors + 1):
                if kx == 0 and ky == 0 and kz == 0:
                    continue

                # Construct the wave vector
                kvec = np.array([kx, ky, kz])
                rkvec = boxUnitCircle * kvec

                # Real-space wave vector and its square
                rksq = np.dot(rkvec, rkvec)

                # Compute the exponential damping factor
                damping = np.exp(-0.25 * rksq / alpha**2) / rksq

                # Adjust the sign for negative kz
                eikz_temp = np.copy(eik[abs(kz), :, 2])
                eikz_temp.imag *= np.sign(kz)

                # Compute the Fourier transform of the charge density
                cksum = np.sum(charges * eik_xy * eikz_temp)

                # Accumulate energy contribution
                energy += doubleForKX * damping * np.abs(cksum) ** 2

    # Multiply by the prefactor and normalize
    energy *= 2.0 * np.pi / volume
    return energy / numberOfAtoms


def selfEnergy(charges: np.ndarray, alpha: float):
    """
    Compute the self-energy correction for the Ewald summation.

    Parameters:
    charges (np.ndarray): Array of charges.
    alpha (float): The Ewald damping parameter.

    Returns:
    float: The self-energy correction per atom.
    """
    return -alpha * np.sum(charges**2) / (np.sqrt(np.pi) * charges.shape[0])


def totalEnergy(
    positions: np.ndarray, charges: np.ndarray, boxSize: float, alpha: float, numberOfWaveVectors: int
) -> np.float64:
    """
    Compute the total energy per atom using the Ewald summation.

    Parameters:
    positions (np.ndarray): Array of atom positions.
    charges (np.ndarray): Array of charges.
    boxSize (float): The size of the simulation box.
    alpha (float): The Ewald damping parameter.
    numberOfWaveVectors (int): The maximum number of wave vectors to consider in each direction.

    Returns:
    float: The total energy per atom.
    """
    real = realSpace(positions, charges, boxSize, alpha)
    fourier = fourierSpace(positions, charges, boxSize, alpha, numberOfWaveVectors)
    return real + fourier + selfEnergy(charges, alpha)

In [19]:
alpha = 1.0
numberOfWaveVectors = 32
boxSize = 20
charge = 1.0

positions, charges = getPositionsCharges(boxSize, charge)

In [20]:
totalEnergy(positions, charges, boxSize, alpha, numberOfWaveVectors)

-0.8737822973162099